# Collecting data from ThermoML xml files

In [ ]:
import os
from ThermoML import MyThermoMLBuilder
import polars as pl
import numpy as np
from importlib import reload

## Phase Equilibria Composition

Collecting data from files with phase equilibria composition data

In [ ]:
dir = os.getcwd()
files = []
with open('binaryPE.txt', 'r') as f:
    files = [os.path.join(dir, 'ThermoML', file.strip()+'.xml')
             for file in f.readlines()]


In [ ]:
files[0]


In [ ]:
property = ['Mole fraction']
[data, compound_dict] = build_dataset(files, property)


In [ ]:
data.replace(['None', 'nan', "NaN"], np.nan, inplace=True)


In [ ]:
data.to_parquet('datasetPE.parquet', engine='pyarrow', index=False)


In [ ]:
compound_dict.to_json('compounddictPE.json', orient='index')


## Activity coeficient

Collecting data from files with activity coeficient at infinity dilution data

In [26]:
reload(MyThermoMLBuilder)

<module 'ThermoML.MyThermoMLBuilder' from '/home/wildsonbbl/documents/code/themoml/ThermoML/MyThermoMLBuilder.py'>

In [25]:
dir = os.getcwd()
files = []
with open('binaryAC.txt', 'r') as f:
    files = [os.path.join(dir, 'ThermoML', file.strip()+'.xml')
             for file in f.readlines()]
files[0]


'/home/wildsonbbl/documents/code/themoml/ThermoML/10.1016/j.fluid.2014.11.020.xml'

In [27]:
property = ['Activity coefficient', 'Mole fraction', "Mass fraction"]
[data, compounds] = MyThermoMLBuilder.build_dataset(files, property)


In [31]:
compounds.describe()

describe,CommonName,StandardInChI
str,str,str
"""count""","""568""","""568"""
"""null_count""","""0""","""0"""
"""mean""",null,null
"""std""",null,null
"""min""","""(1-methylethen...","""InChI=1S/2C8H1..."
"""max""","""water""","""InChI=1S/Sb"""
"""median""",null,null


In [30]:
data.describe()

describe,filename,nDATA,c1,c2,phase_1,phase_2,"Pressure, kPa","Temperature, K",Mole fraction c1 phase_2,Activity coefficient c1 phase_2,Mole fraction c1 phase_1,Activity coefficient c1 phase_1,Mole fraction c2 phase_1,Activity coefficient c2 phase_1,Mole fraction c2 phase_2,Activity coefficient c2 phase_2,c3,Mass fraction c3 phase_2,Activity coefficient c3 phase_2,Mole fraction c3 phase_1,Mole fraction c3 phase_2,Mass fraction c2 phase_2,Mass fraction c1 phase_1
str,str,f64,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64
"""count""","""44367""",44367.0,"""44367""","""44367""","""44367""","""44367""",44367.0,44367.0,44367.0,44367.0,44367.0,44367.0,44367.0,44367.0,44367.0,44367.0,"""44367""",44367.0,44367.0,44367.0,44367.0,44367.0,44367.0
"""null_count""","""0""",0.0,"""0""","""0""","""0""","""31048""",12544.0,299.0,41139.0,41505.0,38240.0,38458.0,27665.0,28027.0,39105.0,39422.0,"""43550""",44339.0,44284.0,43856.0,44305.0,44343.0,44362.0
"""mean""",null,43.531566,null,null,null,null,100.210977,335.31985,0.060007,15.170093,0.026608,321.072321,0.021943,28.09914,0.026562,49.914834,null,0.0,5.668759,0.220034,0.111603,0.0,0.0
"""std""",null,70.398038,null,null,null,null,14.038305,27.514769,0.196327,100.91524,0.139287,5248.237315,0.126555,186.726843,0.133233,255.843514,null,0.0,9.802903,0.2962,0.315385,0.0,0.0
"""min""","""/home/wildsonb...",1.0,"""(1-methylethen...","""(chloromethyl)...","""Air at 1 atmos...","""Liquid""",0.45,249.84,0.0,0.0,0.0,0.022,0.0,0.0156,0.0,0.0373,"""1-pentene""",0.0,0.146,0.0,0.0,0.0,0.0
"""max""","""/home/wildsonb...",594.0,"""vinyl acetate""","""water""","""Liquid mixture...","""Liquid mixture...",238.0,1200.0,1.0,2447.0,1.0,149008.0,1.0,9781.0,1.0,9813.0,"""water""",0.0,43.0,1.0,0.9911,0.0,0.0
"""median""",null,25.0,null,null,null,null,101.0,333.15,0.0,2.23,0.0,2.07,0.0,2.55,0.0,3.94,null,0.0,2.22,0.1119,0.0,0.0,0.0


In [38]:
data.write_parquet('datasetAC.parquet')


In [39]:
compounds.write_parquet('compoundsAC.parquet')


In [40]:
compounds = pl.read_parquet('compoundsAC.parquet')


In [41]:
compounds.filter(pl.col('CommonName') == 'choline chloride')


CommonName,StandardInChI
str,str
"""choline chlori...","""InChI=1S/C5H14..."
